# Технические детали

In [1]:
import json
import requests
import xml.dom.minidom as minidom
import datetime as dt

In [2]:
headers = {'Pragma': 'no-cache',
           'Origin': 'http://ibg.dev.b2bpolis.ru/',
           'Accept-Encoding': 'gzip, deflate',
           'Accept-Language': 'en-US,en;q=0.8,ru;q=0.6',
           'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36',
           'Content-Type': 'application/json;charset=UTF-8',
           'Accept': 'application/json, text/plain, */*',
           'Cache-Control': 'no-cache',
           'Connection': 'keep-alive',
           'Authorization': # поставить Токен авторизации,
           'Cookie':        # поставить Куки
           }

# Создаём участников страхования

In [3]:
url_person = 'http://ibg.dev.b2bpolis.ru/rest/default/client/natural-person-create'
METHOD     = 'POST'

### Преобразование адреса человека к нужному формату

РСА требует наличие адреса. Мы используем сервис dadata.ru

In [4]:
url_addr  = 'https://dadata.ru/api/v2/suggest/address'
METHOD    = 'POST'
DATA_addr =
u"""
{
    "query": "Москва, ул Молодёжная, д 48, кв 5"
}
"""

Если всё хорошо и dadata понял нас, в ответе придёт очень много информации, но нас интересует только малая часть

In [5]:
response_addr = requests.post(url=url_addr, headers=headers, data=DATA_addr.encode('utf8'))
parsed_resp_addr = json.loads(response_addr.text)
addr_data = parsed_resp_addr['suggestions'][0]['data']

address =   {"address_type": 1,                         # ДОБАВЛЯЕМ НЕОБХОДИМОЕ ПОЛЕ          
             "country"     : addr_data['country'],
             "city"        : addr_data['city'],
             "street"      : addr_data["street"],
             "house"       : addr_data["house"],
             "flat"        : addr_data["flat"],
             "kladr"       : addr_data["kladr_id"],
             }
print json.dumps(address, indent=4, sort_keys=True).decode('unicode-escape')

{
    "address_type": 1, 
    "city": "Москва", 
    "country": "Россия", 
    "flat": "5", 
    "house": "48", 
    "kladr": "77000000000189000", 
    "street": "Молодёжная"
}


## - водителя(ей)

для каждого водителя делаем свой запрос

In [6]:
DATA_driver =
u"""
{
    "address": [
        {
            "address_type": 1, 
            "city": "Москва", 
            "country": "Россия", 
            "flat": 5, 
            "house": "48", 
            "kladr": "77000000000189000", 
            "street": "Молодёжна"
        }
    ], 
    "birth_date": "1986-12-14", 
    "check_driver": "true",                   # ДОБАВЛЯЕМ НЕОБХОДИМОЕ ДЛЯ ВОДИТЕЛЕЙ ПОЛЕ
    "credential": [
        {
            "credential_type": 3,             # НЕОБХОДИМОЕ ПОЛЕ
            "issue_point": "РФ", 
            "number": "191882", 
            "series": "16ТН"
        }
    ], 
    "driving_experience_started": "2008-11-08", 
    "first_name": "Марина", 
    "last_name": "Бердникова", 
    "patronymic": "Николаевна"
}
"""

In [7]:
response_driver = requests.post(url=url_person, data=DATA_driver.encode('utf8'), headers=headers)
parsed_resp_driver = json.loads(response_driver.text)

# рассчитываем возраст и стаж водителя
driver_exp = dt.datetime.now().year - int(parsed_resp_driver['driving_experience_started'][:4])
driver_age = (dt.datetime.now() - dt.datetime.strptime(parsed_resp_driver['birth_date'], '%Y-%m-%d')).days/365
print 'driver age:', driver_exp
print 'driver exp:', driver_age

# id водителя
driver_idx = parsed_resp_driver['id']
print 'driver id:', driver_idx

driver age: 8
driver exp: 29
driver id: 3498


## - собственника, страхователя

делаем ещё раз запрос адреса по предыдущей схеме, который и помещаем в запрос Собственника

In [8]:
DATA_owner =
u"""
{
    "address": [
        {
            "address_type": 1, 
            "city": "Канаш", 
            "country": "Россия", 
            "flat": null, 
            "house": "54", 
            "kladr": "2100002300000450003", 
            "street": "Комсомольская"
        }
    ], 
    "birth_date": "1984-03-14", 
    "check_owner": true,                        # ДОБАВЛЯЕМ НЕОБХОДИМОЕ ДЛЯ СОБСТВЕННИКА ПОЛЕ
    "credential": [
        {
            "credential_type": 1,               # НЕОБХОДИМОЕ ПОЛЕ
            "issue_date": "2006-02-28", 
            "issue_point": "УФМС", 
            "number": "412777", 
            "series": "9705"
        }
    ], 
    "first_name": "Рашид", 
    "gender": "M", 
    "last_name": "Гафиятуллин", 
    "patronymic": "Минсеитович"
}
"""

In [9]:
response_owner = requests.post(url=url_person, data=DATA_owner.encode('utf8'), headers=headers)
parsed_resp_owner = json.loads(response_owner.text)

# id собственника
owner = parsed_resp_owner['person']
print 'owner:', owner

owner: 3178


# Создаём страхуемое авто

## - получаем id марки и модели авто

In [10]:
url_car_mark = 'http://ibg.dev.b2bpolis.ru/rest/full/car_mark/'
METHOD       = 'GET'

In [11]:
car_marks = {}
response_car_mark = requests.get(url=url_car_mark, headers=headers)
parsed_resp_car_mark = json.loads(response_car_mark.text)
for row in parsed_resp_car_mark:
    is_for, idx, title = row.values()
    car_marks[title] = idx

In [12]:
# выбираем марку
car_mark = car_marks['Skoda']
car_mark

65176

In [13]:
url_car_models = 'http://ibg.dev.b2bpolis.ru/rest/full/car_mark/{}/car_model'.format(car_mark)
METHOD         = 'GET'

In [14]:
car_models = {}
response_car_models = requests.get(url=url_car_models, headers=headers)
parsed_resp_car_models = json.loads(response_car_models.text)
for row in parsed_resp_car_models:
    title, idx, _, _ = row.values()
    car_models[title] = idx

In [15]:
# выбираем модель
car_model = car_models['FABIA']
car_model

65190

## - создаём авто

In [16]:
url_car = 'http://ibg.dev.b2bpolis.ru/rest/default/client/car-create'
METHOD  = 'POST'

In [17]:
DATA_car =
u"""
{
    "car_mark": 65176, 
    "car_model": 65190, 
    "car_modification": null,                             # НЕОБХОДИМОЕ ПОЛЕ 
    "cost": 0,                                            # НЕОБХОДИМОЕ ПОЛЕ 
    "credential": [
        {
            "credential_type": 4,                         # НЕОБХОДИМОЕ ПОЛЕ 
            "issue_date": "2011-07-19", 
            "number": "850677", 
            "series": "77УН"
        }
    ], 
    "engine_power": 105, 
    "engine_power_kilowatt": 77.23, 
    "manufacturing_date": "2011-01-01", 
    "mileage": "20", 
    "number_plate": "Е271ХМ178", 
    "vin_number": "TMBED45J2B3209311"
}
"""

In [18]:
response_car = requests.post(url=url_car, data=DATA_car.encode('utf8'), headers=headers)
parsed_resp_car = json.loads(response_car.text)

car_idx = parsed_resp_car['id']
print 'car id:', car_idx

car id: 4448


# Создаём расчёт

## - собираем всё в один объект

In [19]:
url_ins_obj = 'http://ibg.dev.b2bpolis.ru/rest/default/client/insured-object'
METHOD      = 'POST'

In [20]:
# все id водителей объединяем в массив
DATA_ins_obj =
u"""
{
    "beneficiary": 3174,                          # owner
    "drivers": [
        3494                                      # driver_idx
    ], 
    "insurant": 3174,                             # owner
    "object_id": 4446,                            # car_idx 
    "object_type": "car", 
    "owner": 3174                                 # owner
}
"""

In [21]:
response_ins_obj = requests.post(url=url_ins_obj, data=DATA_ins_obj.encode('utf8'), headers=headers)
parsed_resp_ins_obj = json.loads(response_ins_obj.text)

idx_ins_obj = parsed_resp_ins_obj['id']
print 'insurable_object id:', idx_ins_obj

insurable_object id: 4223


## - создаём расчёт

In [22]:
url_calc = 'http://ibg.dev.b2bpolis.ru/rest/full/calculation/'
METHOD   = 'POST'

In [23]:
DATA_calc =
u"""
{
    "calculation_at_date": "2016-11-09",                # дата "сегодня"
    "car_cost": "0",                                    # НЕОБХОДИМОЕ ПОЛЕ
    "car_manufacturing_year": 2011, 
    "car_mark": 65176, 
    "car_model": "65190", 
    "car_type": 46202,                                  # НЕОБХОДИМОЕ ПОЛЕ
    "created_at": "2016-11-07T18:44:03.047",            # дата "сегодня"
    "driver_set": [
        {
            "age": 29,                                  # инфа о водителях в массиве
            "expirience": 8
        }
    ], 
    "drivers_count": 1,                                 # кол-во водителей
    "drivers_minimal_age": 29, 
    "drivers_minimal_experience": 7, 
    "engine_power": 105,
    "exploitation_area": 50906,                         # НЕОБХОДИМОЕ ПОЛЕ 
    "has_foreign_registration": false,                  # ТС зарегистрирован в другой стране
    "insurance_duration": 46227,                        # НЕОБХОДИМОЕ ПОЛЕ 
    "insurance_type": 37564,                            # НЕОБХОДИМОЕ ПОЛЕ 
    "insurant_type": 53666,                             # НЕОБХОДИМОЕ ПОЛЕ 
    "insured_object": 4221,                             # idx_ins_obj
    "is_calculate_osago": true,                         # НЕОБХОДИМОЕ ПОЛЕ
    "is_legal_entity": false,                           # юридическое лицо
    "is_multidrive": false,                             # мультидрайв
    "is_new_car": false,                                # новый автомобиль
    "is_prolongation": false,                           # пролонгация
    "is_seasonal_using": false,                         # сезонное использование
    "is_without_registration_car": false,               # ТС без регистрации
    "owner_registration": 46239,                        # НЕОБХОДИМОЕ ПОЛЕ
    "policy_area": 46595,                               # НЕОБХОДИМОЕ ПОЛЕ
    "prolongation": 51028,                              # НЕОБХОДИМОЕ ПОЛЕ
    "target_of_using": 46596,                           # НЕОБХОДИМОЕ ПОЛЕ
    "using_api": true                                   # НЕОБХОДИМОЕ ПОЛЕ
}
"""

In [24]:
response_calc = requests.post(url=url_calc, data=DATA_calc.encode('utf8'), headers=headers)
parsed_resp_calc = json.loads(response_calc.text)
idx_calc = parsed_resp_calc['id']
print 'id расчёта:', idx_calc

id расчёта: 6651226


## - ещё один технический зарос без данных

In [25]:
url_first_final = 'http://ibg.dev.b2bpolis.ru/rest/v3/default/calculation/{}/result/67405/?insurance_company=1&messages=1&vars=1'.format(idx_calc)
METHOD          = 'POST'

In [26]:
response_first_final = requests.post(url=url_first_final, headers=headers)
parsed_resp_first_final = json.loads(response_first_final.text)

result_idx = parsed_resp_first_final[0]['id']
print result_idx

68306443


# Получаем полис

## - создаём полис

In [27]:
url_res_pol = 'http://ibg.dev.b2bpolis.ru/policy/rest/result_policy/'
METHOD      = 'POST'

In [28]:
req_res_pol =
u"""
{
    "exploitation_period_begin_in_contract_time": "2016-11-09", 
    "exploitation_period_end_in_contract_time": "2017-11-08", 
    "insured_object": 4221, 
    "result": 68306441, 
    "status": "saved", 
    "valid_from": "2016-11-09T0:00", 
    "valid_to": "2017-11-08T23:59"
}
"""

In [29]:
response_res_pol = requests.post(url=url_res_pol, data=req_res_pol.encode('utf8'), headers=headers)
parsed_resp_res_pol = json.loads(response_res_pol.text)

policy_idx = parsed_resp_res_pol['id']
print 'id полиса:', policy_idx

id полиса: 1258


## - получаем электронные версии полиса, анкеты и заявления

In [30]:
url_print = 'http://ibg.dev.b2bpolis.ru/policy/rest/result_policy/{}/print/'.format(policy_idx)
METHOD    = 'POST'

In [31]:
response_print = requests.post(url=url_print, headers=headers)
parsed_resp_print = json.loads(response_print.text)

print 'http://ibg.dev.b2bpolis.ru/policy/rest/result_policy/{}/print/1/'.format(policy_idx)
print 'http://ibg.dev.b2bpolis.ru/policy/rest/result_policy/{}/print/2/'.format(policy_idx)
print 'http://ibg.dev.b2bpolis.ru/policy/rest/result_policy/{}/print/3/'.format(policy_idx)

http://ibg.dev.b2bpolis.ru/policy/rest/result_policy/1258/print/1/
http://ibg.dev.b2bpolis.ru/policy/rest/result_policy/1258/print/2/
http://ibg.dev.b2bpolis.ru/policy/rest/result_policy/1258/print/3/
